In [ ]:
import sys
sys.path.append('/mnt/efs/home/paul/lumos-ncpt-tools') # REMOVE!
from lumos_ncpt_tools import NCPT, load_data
import numpy as np
import pandas as pd

# Demo functionality of the lumos-ncpt-tools module
### TODO: Check to see if data can be loaded on standard computer... 

### 1) Load up some data! 
- Instantiate a NCPT class object <br>
- Display some info about the battery data <br>
- Display info about the subtests in the battery <br>

### 2) Process and plot the scores 
- Check for outlier scores <br>
- Lookup normalized scores and the Grand Index (a composite score) <br>
    - Lookup both types of normalized score? <br>
- Plot the normalized scores vs. age <br>

### 3) Miscellaneous 
- Filter out users who did not complete the test battery <br>
- Show the meaning of the education levels <br>
- Save the filtered dataset <br>

### Load up some data

In [ ]:
# Load data
data_directory = 'CHANGE/TO/DATA/DIRECTORY' 
data_fn = 'CHANGE/TO/DATA/FILENAME' 
df = load_data(data_path, raw_fn)

# Instantiate a NCPT object, get some basic info
ncpt = NCPT(df)
del df
ncpt.report_stats()

# Keep only the tests from a specific test battery
# TODO: Incomplete
keep_battery = 39